<a href="https://colab.research.google.com/github/BNkosi/Zeus/blob/master/Zeus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zeus.py

## Contents
1. First installation
2. Imports
3. Data
4. Data cleaning and Preprocessing
5. Retriever
6. Reader
7. Finder
8. Prediction

In [ ]:
# First instalation
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
# Make sure you have a GPU running
!nvidia-smi

Mon Aug 31 08:48:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Imports

In [4]:
# Minimum imports
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.faiss import FAISSDocumentStore
from haystack.retriever.dense import DensePassageRetriever

08/31/2020 08:48:40 - INFO - faiss -   Loading faiss with AVX2 support.
08/31/2020 08:48:40 - INFO - faiss -   Loading faiss.


## Load Data

In [5]:
def fetch_data_from_repo(doc_dir = "data5/website_data/", 
                         s3_url = "https://github.com/Thabo-5/Chatbot-scraper/raw/master/txt_files.zip",
                         doc_store=FAISSDocumentStore()):
    """
    Function to download data from s3 bucket/ github
    Parameters
    ----------
        doc_dir (str): path to destination folder
        s3_url (str): path to download zipped data
        doc_store (class): Haystack document store
    Returns
    -------
        document_store (object): Haystack document store object
    """
    document_store=doc_store
    fetch_archive_from_http(url=s3_url, output_dir=doc_dir)
    import os
    for filename in os.listdir(path=doc_dir):
        with open(os.path.join(doc_dir, filename), 'r', encoding='utf-8', errors='replace') as file:
            text = file.read()
            file.close()
        with open(os.path.join(doc_dir, filename), 'w', encoding='utf-8', errors='replace') as file:
            file.write(text)
            file.close()
    # Convert files to dicts
    dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

    # Now, let's write the dicts containing documents to our DB.
    document_store.write_documents(dicts)
    return document_store

In [6]:
document_store = fetch_data_from_repo()

08/31/2020 08:48:49 - INFO - haystack.indexing.utils -   Fetching from https://github.com/Thabo-5/Chatbot-scraper/raw/master/txt_files.zip to `data5/website_data/`
100%|██████████| 102378/102378 [00:00<00:00, 3047150.55B/s]


## Initialize Retriver, Reader and Finder

In [7]:
def initFinder():
    """
    Function to initiate retriever, reader and finder
    Parameters
    ----------
    Returns
    -------
        finder (object): Haystack finder
    """
    retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  use_gpu=False,
                                  embed_title=True,
                                  max_seq_len=256,
                                  batch_size=16,
                                  remove_sep_tok_from_untitled_passages=True)
    # Important: 
    # Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
    # previously indexed documents and update their embedding representation. 
    # While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
    # At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
    document_store.update_embeddings(retriever)
    reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
    return Finder(reader, retriever)

In [8]:
finder = initFinder()

08/31/2020 08:48:58 - INFO - filelock -   Lock 140300911750448 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


08/31/2020 08:48:59 - INFO - filelock -   Lock 140300911750448 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


08/31/2020 08:49:00 - INFO - filelock -   Lock 140303357000672 acquired on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


08/31/2020 08:49:01 - INFO - filelock -   Lock 140303357000672 released on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


08/31/2020 08:49:01 - INFO - filelock -   Lock 140303357000672 acquired on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


...

08/31/2020 08:51:45 - INFO - filelock -   Lock 140300910442648 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


08/31/2020 08:51:46 - INFO - filelock -   Lock 140300910442648 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


08/31/2020 08:51:46 - INFO - filelock -   Lock 140300910442648 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


08/31/2020 08:51:47 - INFO - filelock -   Lock 140300910442648 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


08/31/2020 08:51:48 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
08/31/2020 08:51:48 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
08/31/2020 08:51:48 - INFO - farm.infer -    0 
08/31/2020 08:51:48 - INFO - farm.infer -   /w\
08/31/2020 08:51:48 - INFO - farm.infer -   /'\
08/31/2020 08:51:48 - INFO - farm.infer -   


In [9]:
def getAnswers(retrieve=3, read=5, num_answers=1):
    while(True):
        query = input("You: ")
        if query == "bye":
            print("Goodbye!")
            break
        prediction = finder.get_answers(question=query, top_k_retriever=retrieve, top_k_reader=read)
        for i in range(0, num_answers):
            print(f"\nAnswer\t: {prediction['answers'][i]['answer']}")
            print(f"Context\t: {prediction['answers'][i]['context']}")
            print(f"Document name\t: {prediction['answers'][i]['meta']['name']}")
            print(f"Probability\t: {prediction['answers'][i]['probability']}\n\n")

In [ ]:
getAnswers()

You: When  is the next data science course?


08/28/2020 19:58:20 - INFO - haystack.finder -   Reader is looking for detailed answer in 102356 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:49<00:00, 24.98s/ Batches]


Answer	: Jul 06, 2020

Context	: ip yourself for the future. Learn the skills that matter.
Paragraph: Jul 06, 2020 - Sep 30, 2020
Paragraph: Applications will open again in the future

Document name	: datascience-for-highschool.txt

Probability	: 0.6954488987506536


You: When is the next data engineering course?


08/28/2020 20:01:59 - INFO - haystack.finder -   Reader is looking for detailed answer in 140677 chars ...
Inferencing Samples: 100%|██████████| 2/2 [01:05<00:00, 32.74s/ Batches]


Answer	: Jul 05, 2021

Context	: ascience.net/long-courses/data-engineering
Paragraph: Jan 13, 2021 - Jul 05, 2021
Paragraph: Applications will open again in the future
H3: Explore th

Document name	: data-engineering.txt

Probability	: 0.7017886766075356


You: how do i apply?


08/28/2020 20:05:19 - INFO - haystack.finder -   Reader is looking for detailed answer in 12646 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.44s/ Batches]


Answer	: via our website

Context	:  long). Competition for these places is 
A: All applications happen via our website. Select the course and location (on campus or online) which works 

Document name	: faq.txt

Probability	: 0.7067321198226408


You: bye
Goodbye!


In [ ]:
getAnswers(5,3,1)

You: When is the next data science course?


08/31/2020 08:53:14 - INFO - haystack.finder -   Reader is looking for detailed answer in 213380 chars ...
Inferencing Samples: 100%|██████████| 2/2 [01:07<00:00, 33.55s/ Batches]



Answer	: Jul 06, 2020 - Jun 30, 2021
Context	: roblems using the latest advances in Data Science.
Paragraph: Jul 06, 2020 - Jun 30, 2021
Paragraph: Applications will open again in the future
H3: Jo
Document name	: data-science.txt
Probability	: 0.7106661747683017


